In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch

In [3]:
train = pd.read_csv(r'C:\Users\saivi\Downloads\mnist\train.csv')


test = pd.read_csv(r'C:\Users\saivi\Downloads\mnist\test.csv')

In [4]:
y_train = train['label']
train.drop(columns=['label'], axis=1, inplace=True)

In [5]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Normalize((0.5,), (0.5,))
])

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_CV, y_train, y_CV = train_test_split(train, y_train, test_size=0.2)

In [7]:
from torch.utils.data import Dataset, DataLoader
import torch

class CustomImageDataset(Dataset):
    def __init__(self, data, label=None, transform=None):
        self.data = torch.tensor(data.values, dtype=torch.float32).reshape(-1, 1, 28, 28)
        if label is not None:
            self.labels = torch.tensor(label.values, dtype=torch.int64)
        else:
            self.labels = None
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        if self.transform:
            image = self.transform(image)
        if self.labels is not None:
            label = self.labels[idx]
            return image, label
        return image

# Example transformations
transform = transforms.Compose([
    transforms.Normalize((0.5,), (0.5,))
])

# Assuming 'X_train', 'X_CV', 'y_train', 'y_CV', and 'test' are defined DataFrames
train_data = CustomImageDataset(data=X_train, label=y_train, transform=transform)
validation_data = CustomImageDataset(data=X_CV, label=y_CV, transform=transform)
test_data = CustomImageDataset(data=test, transform=transform)  # No labels for test data

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)



In [8]:
import torch.nn as nn
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.tanh(self.conv1(x))
        x = nn.functional.avg_pool2d(x, 2)
        x = torch.tanh(self.conv2(x))
        x = nn.functional.avg_pool2d(x, 2)
        x = x.view(-1, 16 * 4 * 4)
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet5()
error = nn.CrossEntropyLoss()

# SGD Optimizer
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [9]:
import torch.optim as optim
# Check for CUDA (GPU) availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training variables
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

# Specify the number of epochs
num_epochs = 10

for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        
        # Move tensors to the configured device
        data, target = data.to(device), target.to(device)
        
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(data)
        
        # Calculate loss
        loss = error(outputs, target)
        
        # Backpropagation
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
        if count % 50 == 0:
            # Calculate Accuracy on validation data
            correct = 0
            total = 0
            with torch.no_grad():
                for val_images, val_labels in validation_loader:
                    val_images, val_labels = val_images.to(device), val_labels.to(device)
                    outputs = model(val_images)
                    predicted = torch.max(outputs.data, 1)[1]
                    total += len(val_labels)
                    correct += (predicted == val_labels).sum().item()
                
            accuracy = 100 * correct / float(total)
            
            # Store loss and iteration values
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            
            if count % 500 == 0:
                # Print Loss and Accuracy
                print(f"Iteration: {count}  Loss: {loss.item()}  Accuracy: {accuracy}%")


Iteration: 500  Loss: 0.3349838852882385  Accuracy: 95.27380952380952%
Iteration: 1000  Loss: 0.12121649086475372  Accuracy: 96.75%
Iteration: 1500  Loss: 0.08380039036273956  Accuracy: 97.60714285714286%
Iteration: 2000  Loss: 0.03573911637067795  Accuracy: 97.36904761904762%
Iteration: 2500  Loss: 0.09958624839782715  Accuracy: 97.57142857142857%
Iteration: 3000  Loss: 0.01832580752670765  Accuracy: 98.16666666666667%
Iteration: 3500  Loss: 0.032265860587358475  Accuracy: 97.83333333333333%
Iteration: 4000  Loss: 0.024522319436073303  Accuracy: 98.3452380952381%
Iteration: 4500  Loss: 0.062137749046087265  Accuracy: 98.28571428571429%
Iteration: 5000  Loss: 0.013639077544212341  Accuracy: 98.3452380952381%


In [10]:
model.eval()
test_outputs = []

with torch.no_grad():
    for data in test_loader:
        if data is not None:
            data = data.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            test_outputs.extend(predicted.cpu().numpy())



In [11]:
test_predictions = pd.DataFrame(test_outputs, columns=['Label'])
test_predictions['ImageId'] = test_predictions.index + 1
test_predictions = test_predictions[['ImageId', 'Label']]

In [12]:
test_predictions.to_csv('submission.csv', index=False)

In [13]:
torch.save(model.state_dict(), "mnist_digit_recognizer.pth")
